<a href="https://colab.research.google.com/github/lemmingy/My_pipelines/blob/main/Sensor_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top

In [ ]:
pip install tsfresh

センサーデータの変換

In [ ]:
import pandas as pd
import numpy as np
from tsfresh import extract_features

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/Kaggle/Library/

In [ ]:
#サンプル用のDataFrame
df=pd.DataFrame()
df["spectrum_id"]=['000da4633378740f1ee8',
 '000ed1a5a9fe0ad2b7dd',
 '0016e3322c4ce0700f9a',
 '00256bd0f8c6cf5f59c8',
 '003483ee5ae313d37590']
df["spectrum_filename"]=['b2e223339f4abce9b400.dat',
 'e2f150a503244145e7ce.dat',
 '3d58b7ccaee157979cf0.dat',
 'ed3641184d3b7c0ae703.dat',
 '4c63418d39f86dfab9bb.dat']
df

In [ ]:
folder_name="/content/drive/My Drive/Colab Notebooks/Kaggle/Library/Input_data/Sensor_data/spectrum_raw/"

#Tsfreshに入れやすい形にSpectrumデータをロード
def concat_spectrum(df_train, folder_name):

  spec_df=pd.DataFrame()
  for x in df_train.spectrum_filename:
      df_tmp = pd.read_csv(folder_name+x, 
                            sep='\t', 
                            header=None)
      
      #おまけ。513データの物があったため。
      if len(df_tmp)==512:
          df_tmp.drop(index=0,inplace=True)
      df_tmp[0]=x
      spec_df=pd.concat([spec_df, df_tmp])
  return spec_df

spec_df=concat_spectrum(df, folder_name)
spec_df

In [ ]:
#Tsfreshによる特徴量の作成
df_features = extract_features(spec_df, column_id=0)
df_features

# For NN

In [ ]:
#スペクトラムファイルを横持ちで呼び込み

def concat_spectrum2(df_train, folder_name):
  spec_df2=pd.DataFrame()
  for x in df_train.spectrum_filename:
      df_tmp = pd.read_csv(folder_name+x, 
                                sep='\t', 
                                header=None)
      if len(df_tmp[0])==512:
          df_tmp.drop(index=0,inplace=True)
      df_tmp.reset_index(drop=True, inplace=True)
      df_tmp=df_tmp[[1]].T
      df_tmp["spectrum_filename"]=x
      spec_df2=pd.concat([spec_df2, df_tmp])
  return spec_df2

concat_spectrum2(df, folder_name)

In [ ]:
# Conv1Dのシンプルモデル
def mlp_raw():
    model = Sequential()

    kernel=8
    model.add(Conv1D(filters=64, kernel_size=(kernel), strides=1, padding='same',
                     activation='relu', kernel_initializer='he_normal', input_shape=(None, 1)))
    model.add(Conv1D(filters=64, kernel_size=(kernel), strides=1, padding='same',
                     activation='relu', kernel_initializer='he_normal'))
    model.add(Conv1D(filters=64, kernel_size=(kernel), strides=1, padding='same',
                     activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid'))#256
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=64, kernel_size=(kernel), strides=1, padding='same',
                     activation='relu', kernel_initializer='he_normal'))
    model.add(Conv1D(filters=64, kernel_size=(kernel), strides=1, padding='same',
                     activation='relu', kernel_initializer='he_normal'))
    model.add(Conv1D(filters=64, kernel_size=(kernel), strides=1, padding='same',
                     activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid'))#128
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=64, kernel_size=(kernel), strides=1, padding='same',
                     activation='relu', kernel_initializer='he_normal'))
    model.add(Conv1D(filters=64, kernel_size=(kernel), strides=1, padding='same',
                     activation='relu', kernel_initializer='he_normal'))
    model.add(Conv1D(filters=64, kernel_size=(kernel), strides=1, padding='same',
                     activation='relu', kernel_initializer='he_normal'))
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid'))#64
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GlobalAveragePooling1D())
    model.add(Flatten())
    model.add(Dense(units=32, activation='relu',kernel_initializer='he_normal'))    
    model.add(Dense(1,activation="sigmoid"))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=["binary_crossentropy"])
    model.summary()
    return model

In [ ]:
# NNにFitする
def nn_raw_fit_pred(df_train_x,df_train_y,df_test_tmp,model_descript):
    oof_train = np.zeros((df_train_x.shape[0],1))
    oof_test = np.zeros((df_test_tmp.shape[0],1))
    scores=[]
    def expand_dim(x):
        x = np.expand_dims(x,axis=2)
        return x
    print(model_descript)
    df_test_tmp=expand_dim(df_test_tmp)
   
    for i, (train_ix, test_ix) in enumerate(kfold.split(df_train_x,df_train_y)):
        X_tr, y_tr = df_train_x.iloc[train_ix].values,df_train_y.iloc[train_ix].values
        X_val, y_val = df_train_x.iloc[test_ix].values, df_train_y.iloc[test_ix].values
        

        
        X_tr=expand_dim(X_tr)
        X_val=expand_dim(X_val)
      
        print(X_val.shape)
        print(df_test_tmp.shape)
        
        filepath = "mlp_best_model.hdf5" 

        es = EarlyStopping(patience=5, mode='min', verbose=1) 

        checkpoint = ModelCheckpoint(monitor='val_loss',filepath=filepath, save_best_only=True, mode='auto') 

        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.1, verbose=1, mode='min')
        print(X_tr.shape)

        model = mlp_raw(X_tr)

        history = model.fit(X_tr, y_tr, batch_size=32, epochs=100, validation_data=(X_val, y_val), 
                            callbacks=[es, checkpoint, reduce_lr_loss], verbose=100)

        y_pred = model.predict(X_val, batch_size=32)
        oof_train[test_ix] = y_pred

        score=log_loss (y_val, y_pred)
        print('CV log_loss Score of Fold_%d is %f' % (i, score))

        score=average_precision_score (y_val, y_pred)
        print('CV average_precision_score of Fold_%d is %f' % (i, score))

        scores.append(score)
        test_pred=model.predict(df_test_tmp, batch_size=32)
        oof_test += test_pred


    print(np.mean(scores),"+/-",np.std(scores))
    
    oof_test /= NFOLDS
    return oof_train,oof_test